## 1、用tushare自选股票，并进行预处理

In [16]:
import tushare as ts
import numpy as np
# 选择股票代码
stock_code = '000001.SZ'  # 以茅台股票为例
ts.set_token("c5f8cb0fb5599edc61902a621f4f3c8b7549c86323936db30c0c259d")
pro=ts.pro_api()
start_date = '20230101'  
end_date = '20231231'  
# 使用Tushare下载股票数据
stock_data = pro.daily(ts_code=stock_code, start_date=start_date, end_date=end_date)
stock_data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20231229,9.42,9.48,9.35,9.39,9.45,-0.06,-0.6349,853852.79,803196.744
1,000001.SZ,20231228,9.11,9.47,9.08,9.45,9.12,0.33,3.6184,1661591.84,1550256.591
2,000001.SZ,20231227,9.10,9.13,9.02,9.12,9.10,0.02,0.2198,641534.35,582036.661
3,000001.SZ,20231226,9.19,9.20,9.07,9.10,9.19,-0.09,-0.9793,541896.33,493746.623
4,000001.SZ,20231225,9.18,9.20,9.14,9.19,9.20,-0.01,-0.1087,413970.88,379638.234
...,...,...,...,...,...,...,...,...,...,...,...
237,000001.SZ,20230109,14.75,14.88,14.52,14.80,14.62,0.18,1.2312,1057659.11,1561368.487
238,000001.SZ,20230106,14.50,14.72,14.48,14.62,14.48,0.14,0.9669,1195744.71,1747915.169
239,000001.SZ,20230105,14.40,14.74,14.37,14.48,14.32,0.16,1.1173,1665425.18,2417272.356
240,000001.SZ,20230104,13.71,14.42,13.63,14.32,13.77,0.55,3.9942,2189682.53,3110729.449


## 2、构建5日移动平均、加权移动平均以及指数加权移动平均函数，并构建类库

In [17]:
import pandas as pd

class MovingAverages:
    def __init__(self, data):
        self.data = data

    def simple_moving_average(self, window=5):
        return self.data['close'].rolling(window=window).mean()

    def weighted_moving_average(self, window=5):
        weights = range(1, window+1)
        return self.data['close'].rolling(window=window).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)

    def exponential_moving_average(self, span=5):
        return self.data['close'].ewm(span=span, adjust=False).mean()


## 3、 用双均线捕捉股票的买卖点，并计算所有买卖点的成功率

In [18]:
def double_moving_average_strategy(data, short_window=5, long_window=20):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0

    # 计算短期均线和长期均线
    signals['short_mavg'] = data['close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['close'].rolling(window=long_window, min_periods=1).mean()

    # 产生买入信号
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)

    # 计算买入点成功率
    buy_signals = signals[signals['signal'] == 1.0]
    success_rate = len(buy_signals[buy_signals['short_mavg'] > buy_signals['long_mavg']]) / len(buy_signals)

    return signals, success_rate


## 4、 用异同均线捕捉股票的买卖点，并计算成功率

In [19]:
def macd_strategy(data, short_window=12, long_window=26, signal_window=9):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0

    # 计算短期和长期指数移动平均线
    exp_short = data['close'].ewm(span=short_window, adjust=False).mean()
    exp_long = data['close'].ewm(span=long_window, adjust=False).mean()

    # 计算MACD线和信号线
    signals['MACD'] = exp_short - exp_long
    signals['signal_line'] = signals['MACD'].ewm(span=signal_window, adjust=False).mean()

    # 产生买入信号
    signals['signal'][short_window:] = np.where(signals['MACD'][short_window:] > signals['signal_line'][short_window:], 1.0, 0.0)

    # 计算买入点成功率
    buy_signals = signals[signals['signal'] == 1.0]
    success_rate = len(buy_signals[buy_signals['MACD'] > buy_signals['signal_line']]) / len(buy_signals)

    return signals, success_rate


In [14]:
# 使用之前定义的移动平均类库
ma = MovingAverages(stock_data)

# 计算5日简单移动平均线
sma_5 = ma.simple_moving_average(window=5)

# 计算20日简单移动平均线
sma_20 = ma.simple_moving_average(window=20)

# 输出简单移动平均线数据检查
print("5日简单移动平均线：")
print(sma_5.head())
print("\n20日简单移动平均线：")
print(sma_20.head())


5日简单移动平均线：
0     NaN
1     NaN
2     NaN
3     NaN
4    9.25
Name: close, dtype: float64

20日简单移动平均线：
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: close, dtype: float64


In [20]:
# 使用双均线策略
signals_ma, success_rate_ma = double_moving_average_strategy(stock_data)
print("双均线策略成功率：", success_rate_ma)

# 使用MACD策略
signals_macd, success_rate_macd = macd_strategy(stock_data)
print("MACD策略成功率：", success_rate_macd)


双均线策略成功率： 1.0
MACD策略成功率： 1.0
